# GIFT Compositional Hierarchy Test with Real L-Function Zeros

**Purpose**: Test the compositional hierarchy hypothesis using actual Dirichlet L-function zeros computed via SageMath.

**Hypothesis**: Composite GIFT conductors (products/sums of primaries) should show better Fibonacci constraint (R ≈ 1) than primary GIFT conductors.

**Predicted hierarchy**:
$$|R - 1|_{\text{composite}} < |R - 1|_{\text{primary}} < |R - 1|_{\text{non-GIFT}}$$

---

## Part 0: Install SageMath on Colab

⚠️ **This takes ~5 minutes** - be patient!

In [ ]:
# Install condacolab (enables conda in Colab)
!pip install -q condacolab
import condacolab
condacolab.install()

⚠️ **After the cell above completes, the runtime will restart. This is normal!**

**Continue with the next cell after restart.**

In [ ]:
# Install SageMath via conda (takes ~3-4 minutes)
!conda install -y -c conda-forge sage=10.2

In [ ]:
# Verify SageMath installation
from sage.all import *
print(f"SageMath version: {version()}")
print("✓ SageMath installed successfully!")

---
## Part 1: Setup and Configuration

In [ ]:
import numpy as np
from scipy import stats
import json
import warnings
warnings.filterwarnings('ignore')

# GIFT Constants
GIFT_LAGS = [5, 8, 13, 27]  # Fibonacci + Jordan

# Conductor Classification
CONDUCTORS = {
    # Composite GIFT (products/sums of primaries) - SHOULD BE BEST
    'composite': {
        6: 'p₂ × N_gen = 2 × 3',
        15: 'N_gen × Weyl = 3 × 5',
        # 16: 'p₂⁴ = 2⁴',  # Skip - not prime conductor
        17: 'dim(G₂) + N_gen = 14 + 3',
        # 99: 'H* = b₂ + b₃ + 1',  # Large - may be slow
    },
    # Primary GIFT (individual topological constants)
    'primary': {
        7: 'dim(K₇)',
        11: 'D_bulk',
        13: 'F₇',
        # 14: 'dim(G₂)',  # Not prime
        # 21: 'b₂',  # Not prime
    },
    # True non-GIFT (primes with no simple GIFT decomposition)
    'non_gift': {
        19: 'Prime (no GIFT)',
        23: 'Prime (no GIFT)',
        29: 'Prime (no GIFT)',
        31: 'Prime (no GIFT)',
    }
}

# Physical observables for composite conductors
PHYSICAL_OBSERVABLES = {
    6: 'sin²θ₂₃(PMNS) = 6/11',
    15: 'Yₚ = 15/61',
    17: 'λ_H = √17/32, σ₈ = 17/21',
}

print("Configuration loaded.")
print(f"\nComposite GIFT: {list(CONDUCTORS['composite'].keys())}")
print(f"Primary GIFT: {list(CONDUCTORS['primary'].keys())}")
print(f"Non-GIFT: {list(CONDUCTORS['non_gift'].keys())}")

---
## Part 2: Compute Dirichlet L-Function Zeros

For each prime conductor q, we compute zeros of L(s, χ) where χ is the primitive quadratic character mod q.

In [ ]:
from sage.all import *

def get_primitive_character(q):
    """
    Get the primitive quadratic character mod q.
    For prime q, this is the Legendre symbol.
    """
    G = DirichletGroup(q)
    for chi in G:
        if chi.is_primitive() and chi.order() == 2:
            return chi
    # If no quadratic character, return first non-trivial primitive
    for chi in G:
        if chi.is_primitive() and not chi.is_trivial():
            return chi
    return None

def compute_zeros_dirichlet(q, num_zeros=100, T_max=100):
    """
    Compute zeros of Dirichlet L-function L(s, χ_q).
    
    Returns list of imaginary parts γ where L(1/2 + iγ, χ) = 0.
    """
    chi = get_primitive_character(q)
    if chi is None:
        print(f"  Warning: No primitive character for q={q}")
        return []
    
    try:
        # Create L-function from character
        L = chi.lfunction()
        
        # Find zeros in [0, T_max]
        zeros = []
        t = 1.0
        step = 0.5
        
        while len(zeros) < num_zeros and t < T_max:
            try:
                # Find zero near t
                z = L.find_zero(t)
                if z is not None and z > 0:
                    # Avoid duplicates
                    if not zeros or abs(z - zeros[-1]) > 0.01:
                        zeros.append(float(z))
            except:
                pass
            t += step
        
        return sorted(zeros)[:num_zeros]
        
    except Exception as e:
        print(f"  Error for q={q}: {e}")
        return []

def compute_zeros_lcalc(q, num_zeros=100):
    """
    Alternative: Use lcalc database if available.
    """
    try:
        chi = get_primitive_character(q)
        if chi is None:
            return []
        
        L = chi.lfunction()
        # Try to get precomputed zeros
        zeros = L.zeros(num_zeros)
        return [float(z) for z in zeros]
    except:
        return compute_zeros_dirichlet(q, num_zeros)

print("Zero computation functions defined.")

In [ ]:
# Test with a single conductor first
print("Testing zero computation for q=7...")
test_zeros = compute_zeros_lcalc(7, num_zeros=20)
print(f"Found {len(test_zeros)} zeros")
if test_zeros:
    print(f"First 5 zeros: {[f'{z:.4f}' for z in test_zeros[:5]]}")

---
## Part 3: Alternative - Use Rubinstein's lcalc

If the above is slow, we can use numerical computation via mpmath as fallback.

In [ ]:
# Fallback: mpmath-based computation for Dirichlet L-functions
import mpmath as mp
mp.mp.dps = 30  # 30 decimal places

def dirichlet_character(n, q):
    """
    Compute quadratic character (Legendre symbol) (n/q) for prime q.
    """
    if n % q == 0:
        return 0
    # Euler's criterion: (n/q) = n^((q-1)/2) mod q
    val = pow(n % q, (q - 1) // 2, q)
    return 1 if val == 1 else -1

def dirichlet_L(s, q, terms=5000):
    """
    Compute L(s, χ_q) where χ_q is the quadratic character mod q.
    """
    result = mp.mpf(0)
    for n in range(1, terms + 1):
        chi_n = dirichlet_character(n, q)
        if chi_n != 0:
            result += chi_n / mp.power(n, s)
    return result

def find_zeros_mpmath(q, num_zeros=50, T_max=80):
    """
    Find zeros of L(1/2 + it, χ_q) by searching for sign changes.
    """
    zeros = []
    t = 1.0
    step = 0.2
    
    prev_val = None
    
    while len(zeros) < num_zeros and t < T_max:
        s = mp.mpc(0.5, t)
        val = dirichlet_L(s, q)
        
        if prev_val is not None:
            # Check for sign change in real part
            if float(prev_val.real) * float(val.real) < 0:
                # Refine with bisection
                t_low, t_high = t - step, t
                for _ in range(20):  # 20 iterations of bisection
                    t_mid = (t_low + t_high) / 2
                    val_mid = dirichlet_L(mp.mpc(0.5, t_mid), q)
                    if float(dirichlet_L(mp.mpc(0.5, t_low), q).real) * float(val_mid.real) < 0:
                        t_high = t_mid
                    else:
                        t_low = t_mid
                zeros.append(float(t_mid))
        
        prev_val = val
        t += step
    
    return zeros

print("mpmath fallback functions defined.")

In [ ]:
# Test mpmath fallback
print("Testing mpmath zero-finding for q=7...")
test_zeros_mp = find_zeros_mpmath(7, num_zeros=10, T_max=50)
print(f"Found {len(test_zeros_mp)} zeros")
if test_zeros_mp:
    print(f"Zeros: {[f'{z:.4f}' for z in test_zeros_mp]}")

---
## Part 4: Compute Zeros for All Conductors

In [ ]:
def get_zeros_for_conductor(q, num_zeros=60, use_sage=True):
    """
    Get zeros for conductor q, trying SageMath first, then mpmath fallback.
    """
    print(f"  Computing zeros for q={q}...", end=" ")
    
    if use_sage:
        try:
            zeros = compute_zeros_lcalc(q, num_zeros)
            if len(zeros) >= num_zeros // 2:
                print(f"SageMath: {len(zeros)} zeros")
                return zeros
        except:
            pass
    
    # Fallback to mpmath
    zeros = find_zeros_mpmath(q, num_zeros, T_max=100)
    print(f"mpmath: {len(zeros)} zeros")
    return zeros

# Collect all conductors
all_conductors = []
for category in CONDUCTORS.values():
    all_conductors.extend(category.keys())
all_conductors = sorted(set(all_conductors))

print(f"Will compute zeros for conductors: {all_conductors}")
print(f"Total: {len(all_conductors)} conductors")

In [ ]:
# Compute zeros for all conductors
# This may take several minutes

NUM_ZEROS = 60  # Number of zeros per conductor
USE_SAGE = True  # Try SageMath first

zeros_by_conductor = {}

print(f"Computing {NUM_ZEROS} zeros for each conductor...\n")

for q in all_conductors:
    zeros = get_zeros_for_conductor(q, NUM_ZEROS, USE_SAGE)
    if len(zeros) >= 30:  # Need enough zeros for recurrence
        zeros_by_conductor[q] = zeros
    else:
        print(f"  ⚠️ Skipping q={q}: only {len(zeros)} zeros")

print(f"\n✓ Computed zeros for {len(zeros_by_conductor)} conductors")

---
## Part 5: Fit Fibonacci Recurrence

In [ ]:
def fit_recurrence(zeros, lags=[5, 8, 13, 27]):
    """
    Fit the recurrence: γ_n = a₅γ_{n-5} + a₈γ_{n-8} + a₁₃γ_{n-13} + a₂₇γ_{n-27} + c
    
    Returns (coefficients, residual_error)
    """
    max_lag = max(lags)
    n_points = len(zeros) - max_lag
    
    if n_points < 10:
        return None, float('inf')
    
    # Build design matrix
    X = np.zeros((n_points, len(lags) + 1))
    y = np.zeros(n_points)
    
    for i in range(n_points):
        idx = i + max_lag
        y[i] = zeros[idx]
        for j, lag in enumerate(lags):
            X[i, j] = zeros[idx - lag]
        X[i, -1] = 1  # constant term
    
    # Solve least squares
    try:
        coeffs, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
        
        # Compute prediction error
        y_pred = X @ coeffs
        error = np.sqrt(np.mean((y - y_pred)**2))
        
        return coeffs, error
    except:
        return None, float('inf')

def compute_fibonacci_ratio(coeffs, lags=[5, 8, 13, 27]):
    """
    Compute R = (8 × a₈) / (13 × a₁₃)
    
    If GIFT structure holds, R ≈ 1.
    """
    if coeffs is None:
        return None
    
    a8 = coeffs[lags.index(8)]
    a13 = coeffs[lags.index(13)]
    
    if abs(a13) < 1e-10:
        return None
    
    R = (8 * a8) / (13 * a13)
    return R

print("Recurrence fitting functions defined.")

In [ ]:
# Compute R for all conductors

results = []

print("Fitting recurrence for each conductor...\n")
print(f"{'q':>4} | {'Category':<12} | {'R':>8} | {'|R-1|':>8} | {'Description'}")
print("-" * 70)

for q in sorted(zeros_by_conductor.keys()):
    zeros = zeros_by_conductor[q]
    
    # Determine category
    if q in CONDUCTORS['composite']:
        category = 'composite'
        desc = CONDUCTORS['composite'][q]
    elif q in CONDUCTORS['primary']:
        category = 'primary'
        desc = CONDUCTORS['primary'][q]
    else:
        category = 'non_gift'
        desc = CONDUCTORS['non_gift'].get(q, 'Unknown')
    
    # Fit recurrence
    coeffs, error = fit_recurrence(zeros, GIFT_LAGS)
    R = compute_fibonacci_ratio(coeffs, GIFT_LAGS)
    
    if R is not None:
        R_dev = abs(R - 1)
        results.append({
            'q': q,
            'category': category,
            'R': float(R),
            'R_deviation': float(R_dev),
            'num_zeros': len(zeros),
            'fit_error': float(error),
            'description': desc
        })
        print(f"{q:>4} | {category:<12} | {R:>8.3f} | {R_dev:>8.3f} | {desc}")
    else:
        print(f"{q:>4} | {category:<12} | {'N/A':>8} | {'N/A':>8} | {desc}")

print("\n✓ Fitting complete")

---
## Part 6: Statistical Analysis

In [ ]:
# Group results by category
composite_results = [r for r in results if r['category'] == 'composite']
primary_results = [r for r in results if r['category'] == 'primary']
non_gift_results = [r for r in results if r['category'] == 'non_gift']

def compute_stats(result_list, name):
    if not result_list:
        print(f"{name}: No data")
        return None, None
    
    deviations = [r['R_deviation'] for r in result_list]
    mean_dev = np.mean(deviations)
    std_dev = np.std(deviations)
    
    print(f"{name}:")
    print(f"  n = {len(result_list)}")
    print(f"  Mean |R - 1| = {mean_dev:.4f} ± {std_dev:.4f}")
    print(f"  Conductors: {[r['q'] for r in result_list]}")
    print()
    
    return mean_dev, std_dev

print("=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
print()

composite_mean, composite_std = compute_stats(composite_results, "COMPOSITE GIFT")
primary_mean, primary_std = compute_stats(primary_results, "PRIMARY GIFT")
non_gift_mean, non_gift_std = compute_stats(non_gift_results, "NON-GIFT")

In [ ]:
# Statistical tests
print("=" * 60)
print("HYPOTHESIS TESTS")
print("=" * 60)
print()

# Test 1: Composite vs Primary
if composite_results and primary_results:
    comp_devs = [r['R_deviation'] for r in composite_results]
    prim_devs = [r['R_deviation'] for r in primary_results]
    
    t_stat, p_value = stats.ttest_ind(comp_devs, prim_devs)
    
    print("Test 1: Composite GIFT vs Primary GIFT")
    print(f"  H₀: No difference in |R - 1|")
    print(f"  H₁: Composite has lower |R - 1|")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value (two-tailed): {p_value:.4f}")
    print(f"  p-value (one-tailed): {p_value/2:.4f}")
    
    if composite_mean < primary_mean:
        print(f"  → Composite mean ({composite_mean:.4f}) < Primary mean ({primary_mean:.4f})")
        if p_value/2 < 0.05:
            print("  ✓ SIGNIFICANT: Composites perform better (p < 0.05)")
        else:
            print("  ○ Not significant at p < 0.05")
    else:
        print(f"  → Composite mean ({composite_mean:.4f}) >= Primary mean ({primary_mean:.4f})")
        print("  ✗ Hypothesis NOT supported")
    print()

# Test 2: Composite vs Non-GIFT
if composite_results and non_gift_results:
    comp_devs = [r['R_deviation'] for r in composite_results]
    non_gift_devs = [r['R_deviation'] for r in non_gift_results]
    
    t_stat, p_value = stats.ttest_ind(comp_devs, non_gift_devs)
    
    print("Test 2: Composite GIFT vs Non-GIFT")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value (two-tailed): {p_value:.4f}")
    
    if composite_mean < non_gift_mean:
        print(f"  → Composite ({composite_mean:.4f}) < Non-GIFT ({non_gift_mean:.4f})")
    print()

In [ ]:
# Rank all conductors by |R - 1|
print("=" * 60)
print("RANKING BY |R - 1| (lower is better)")
print("=" * 60)
print()

sorted_results = sorted(results, key=lambda x: x['R_deviation'])

print(f"{'Rank':>4} | {'q':>4} | {'Category':<12} | {'R':>8} | {'|R-1|':>8} | {'Physical Meaning'}")
print("-" * 80)

for i, r in enumerate(sorted_results, 1):
    phys = PHYSICAL_OBSERVABLES.get(r['q'], '')
    marker = '★' if r['category'] == 'composite' else ' '
    print(f"{i:>4} | {r['q']:>4} | {r['category']:<12} | {r['R']:>8.3f} | {r['R_deviation']:>8.3f} | {phys} {marker}")

# Count composites in top half
n_total = len(sorted_results)
n_top_half = n_total // 2
top_half = sorted_results[:n_top_half]
composites_in_top = sum(1 for r in top_half if r['category'] == 'composite')

print()
print(f"Composites in top half: {composites_in_top}/{len(composite_results)}")

---
## Part 7: Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: |R - 1| by category
ax1 = axes[0]
categories = ['Composite\nGIFT', 'Primary\nGIFT', 'Non-GIFT']
means = [composite_mean or 0, primary_mean or 0, non_gift_mean or 0]
stds = [composite_std or 0, primary_std or 0, non_gift_std or 0]
colors = ['#2ecc71', '#3498db', '#e74c3c']

bars = ax1.bar(categories, means, yerr=stds, capsize=5, color=colors, alpha=0.7)
ax1.set_ylabel('Mean |R - 1|', fontsize=12)
ax1.set_title('Fibonacci Constraint by Category\n(lower is better)', fontsize=14)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Add value labels
for bar, mean in zip(bars, means):
    if mean > 0:
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{mean:.3f}', ha='center', va='bottom', fontsize=11)

# Plot 2: Individual conductors
ax2 = axes[1]

for r in results:
    if r['category'] == 'composite':
        color, marker = '#2ecc71', 's'
    elif r['category'] == 'primary':
        color, marker = '#3498db', 'o'
    else:
        color, marker = '#e74c3c', '^'
    
    ax2.scatter(r['q'], r['R_deviation'], c=color, marker=marker, s=100, alpha=0.7)
    ax2.annotate(str(r['q']), (r['q'], r['R_deviation']), 
                textcoords="offset points", xytext=(5, 5), fontsize=9)

ax2.axhline(y=0, color='green', linestyle='--', alpha=0.5, label='Perfect (R=1)')
ax2.set_xlabel('Conductor q', fontsize=12)
ax2.set_ylabel('|R - 1|', fontsize=12)
ax2.set_title('Fibonacci Deviation by Conductor', fontsize=14)

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='s', color='w', markerfacecolor='#2ecc71', markersize=10, label='Composite GIFT'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', markersize=10, label='Primary GIFT'),
    Line2D([0], [0], marker='^', color='w', markerfacecolor='#e74c3c', markersize=10, label='Non-GIFT'),
]
ax2.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig('compositional_hierarchy_test.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Plot saved as 'compositional_hierarchy_test.png'")

---
## Part 8: Final Verdict

In [ ]:
print("=" * 70)
print("FINAL VERDICT: COMPOSITIONAL HIERARCHY HYPOTHESIS")
print("=" * 70)
print()

# Determine verdict
hierarchy_supported = False
if composite_mean is not None and primary_mean is not None:
    if composite_mean < primary_mean:
        hierarchy_supported = True

print("HYPOTHESIS: |R - 1|_composite < |R - 1|_primary < |R - 1|_non-gift")
print()
print("OBSERVED:")
print(f"  Composite GIFT:  {composite_mean:.4f}" if composite_mean else "  Composite GIFT: N/A")
print(f"  Primary GIFT:    {primary_mean:.4f}" if primary_mean else "  Primary GIFT: N/A")
print(f"  Non-GIFT:        {non_gift_mean:.4f}" if non_gift_mean else "  Non-GIFT: N/A")
print()

if hierarchy_supported:
    print("✓ HYPOTHESIS SUPPORTED")
    print("  Composite GIFT conductors show better Fibonacci constraint")
    print("  than primary GIFT conductors.")
    print()
    print("  INTERPRETATION: Physics emerges from RELATIONS between")
    print("  topological constants, not from the constants themselves.")
else:
    print("✗ HYPOTHESIS NOT SUPPORTED")
    print("  Composite GIFT conductors do NOT show better Fibonacci constraint.")
    print()
    print("  The compositional hierarchy observed in proxy data")
    print("  does not hold for real Dirichlet L-function zeros.")

print()
print("=" * 70)

In [ ]:
# Save results to JSON
output = {
    'hypothesis': 'Compositional hierarchy: |R-1|_composite < |R-1|_primary',
    'data_source': 'Real Dirichlet L-function zeros (SageMath/mpmath)',
    'results': results,
    'summary': {
        'composite_mean': float(composite_mean) if composite_mean else None,
        'primary_mean': float(primary_mean) if primary_mean else None,
        'non_gift_mean': float(non_gift_mean) if non_gift_mean else None,
        'hierarchy_supported': bool(hierarchy_supported)
    }
}

with open('compositional_hierarchy_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Results saved to 'compositional_hierarchy_results.json'")
print()
print("Please share this JSON file to document the findings!")

---

## Summary

This notebook tested the **Compositional Hierarchy Hypothesis**:

> Composite GIFT conductors (products/sums like 6 = 2×3, 17 = 14+3) should show better Fibonacci constraint (R ≈ 1) than primary GIFT conductors (individual constants like 7, 11, 13).

**Physical significance**: Each composite conductor corresponds to a physical observable:
- 6 = p₂ × N_gen → sin²θ₂₃(PMNS) = 6/11
- 15 = N_gen × Weyl → Yₚ = 15/61
- 17 = dim(G₂) + N_gen → λ_H = √17/32, σ₈ = 17/21

If the hierarchy is confirmed, it suggests that **physics emerges from the relational arithmetic of topological constants**, not from the constants themselves.

---

*GIFT Framework — February 2026*